In [9]:
import pandas as pd
import numpy as np

In [10]:
df_외감=pd.read_excel('./Dataset/외감_dp(등급).xls')
df_외감.columns = ['KIS','Stock','회사명','신용평점']

In [11]:
df=pd.read_csv('../외감기업/Dataset/외감raw3.csv',encoding='euc-kr')
df['Year']= df['회계년도'].replace('/','').astype(str).str[:4].astype(int)
df=df[df['Year']==2022]
df['자산(*)(IFRS연결)(천원)'] = df['자산(*)(IFRS연결)(천원)'] * 1000
df['log자산총계'] = np.log(df['자산(*)(IFRS연결)(천원)'])
df['잉여현금흐름(FCF)']=df['영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)']-df['투자활동으로 인한 현금흐름(*)(IFRS연결)(천원)']
df['전체현금흐름(천원)']=df['영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)']+df['투자활동으로 인한 현금흐름(*)(IFRS연결)(천원)']+df['재무활동으로 인한 현금흐름(*)(IFRS연결)(천원)']
df['총자산대비잉여현금흐름'] = df['잉여현금흐름(FCF)']/df['자산(*)(IFRS연결)(천원)']
df['총자산대비현금흐름'] = df['전체현금흐름(천원)'] / df['자산(*)(IFRS연결)(천원)']
df=df[['회사명', '거래소코드','유동자산회전률', '총자산대비잉여현금흐름','자기자본구성비율', 'log자산총계','자기자본회전률', '순운전자본회전률', '자기자본증가율', '총자본증가율', '총자산대비현금흐름', '총자본투자효율']]

c:\Users\dgh06\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [12]:
df_등급화=pd.merge(df,df_외감,how='inner',on='회사명')

In [13]:
df_등급화['신용평점']=df_등급화['신용평점'].map({
    'AAA' : 'AA',
    'AA':'AA',
    'A':'A',
    'BBB':'BBB',
    'BB':'BB',
    'B':'B',
    'CCC':'CCC',
    'CC':'CC',
    'C': 'D',
    'D':'D'

})

In [14]:
df_등급화.isna().sum()

회사명                0
거래소코드              0
유동자산회전률            3
총자산대비잉여현금흐름        3
자기자본구성비율           2
log자산총계            2
자기자본회전률            3
순운전자본회전률           3
자기자본증가율            2
총자본증가율             2
총자산대비현금흐름          3
총자본투자효율            2
KIS                0
Stock          14540
신용평점               0
dtype: int64

In [15]:
df_등급화=df_등급화.dropna()
df_등급화 = df_등급화[(df_등급화 != 0).all(axis=1)]
df_등급화 = df_등급화.replace([np.inf, -np.inf], np.nan)  # inf 값을 NaN으로 대체
df_등급화 = df_등급화.dropna()  # NaN이 있는 행 삭제

In [16]:
import numpy as np

# O-Score의 계수
coefficients = {
    '유동자산회전률':0.47765086,                    # 높아야 좋다
    '총자산대비잉여현금흐름':0.53785436,            # 높아야 좋다
    '자기자본구성비율':0.49360789,                  # 높아야 좋다
    'log자산총계':0.94238478,                      # 높아야 좋다
    '자기자본회전률':-0.08646556,                   # 높아야 좋다
    '순운전자본회전률':-0.98107367,                 # 높아야 좋다
    '자기자본증가율':0.19544705,                    # 높아야 좋다
    '총자본증가율':0.44094258,                      # 높아야 좋다
    '총자산대비현금흐름':-0.37036301,               # 높아야 좋다
    '총자본투자효율':0.26458857,                    # 높아야 좋다

    'constant': 0.90495308
}

# O-Score 계산
df_등급화['dp_score'] = coefficients['constant']
df_등급화['dp_score'] += coefficients['유동자산회전률'] * df_등급화['유동자산회전률']
df_등급화['dp_score'] += coefficients['총자산대비잉여현금흐름'] * df_등급화['총자산대비잉여현금흐름']
df_등급화['dp_score'] += coefficients['자기자본구성비율'] * df_등급화['자기자본구성비율']
df_등급화['dp_score'] += coefficients['log자산총계'] * df_등급화['log자산총계']
df_등급화['dp_score'] += coefficients['자기자본회전률'] * df_등급화['자기자본회전률']
df_등급화['dp_score'] += coefficients['순운전자본회전률'] * df_등급화['순운전자본회전률']
df_등급화['dp_score'] += coefficients['자기자본증가율'] * df_등급화['자기자본증가율']
df_등급화['dp_score'] += coefficients['총자본증가율'] * df_등급화['총자본증가율']
df_등급화['dp_score'] += coefficients['총자산대비현금흐름'] * df_등급화['총자산대비현금흐름']
df_등급화['dp_score'] += coefficients['총자본투자효율'] * df_등급화['총자본투자효율']

# 로지스틱 함수를 이용해 부도 확률 계산
df_등급화['default_probability'] = 1 / (1 + np.exp(-df_등급화['dp_score']))

In [18]:
lower_bound = df_등급화['dp_score'].quantile(0.01)
upper_bound = df_등급화['dp_score'].quantile(0.99)
df_등급화_filtered = df_등급화[(df_등급화['dp_score'] >= lower_bound) & (df_등급화['dp_score'] <= upper_bound)]


In [17]:
df_등급화[['dp_score','신용평점']].groupby('신용평점').agg(['mean', lambda x: x.quantile(0.25), lambda x: x.quantile(0.5), lambda x: x.quantile(0.75)])

dp_score                                 
           mean <lambda_0> <lambda_1> <lambda_2>
신용평점                                            
A     66.789312  54.000230  67.836619  81.421737
AA    69.826705  66.610527  72.144788  78.929001
B     53.369724  44.491386  50.829527  62.240600
BB    66.110675  51.359222  60.893620  68.251813
BBB   62.453246  49.688593  59.825249  73.730508
CC    42.225740  25.388495  41.296266  53.304356
CCC   49.269174  35.486912  42.682812  59.758256
D     83.952231  24.550289  34.637690  94.039632

In [20]:
df_등급화_filtered[['dp_score','신용평점']].groupby('신용평점').mean()

,dp_score
신용평점,
A,68.528125
AA,69.826705
B,53.369724
BB,59.750166
BBB,63.601743
CC,42.225740
CCC,49.269174
D,30.959408
